
# 0 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

import os

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 1 - Data loading

In [ ]:
# Input folder name
inputFolder = 'PAIR_002'
# Concatenate folder name with the path
inputPath = '../data/' + inputFolder + '/'
mesh1 = TriMesh(inputPath + 'target.off', area_normalize=True, center=False)
mesh2 = TriMesh(inputPath + 'source.off', area_normalize=True, center=False)

# 2- Visualization of eigen functions

In [ ]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=100, intrinsic=False, verbose=True);

# set up a figure twice as wide as it is tall
i = 0
d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

# 3 - Computing the functional map

**Displaying data**

In [ ]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])
mp.subplot(mesh2.vertlist, mesh2.facelist, None, s=[2, 2, 1], data=d)

**Add correspondences**

In [ ]:
loadLandmark = True
if loadLandmark:
    landmarks = np.loadtxt(inputPath +'landmarks.txt',dtype=int)[:10]  # loading N landmarks
    display(landmarks)

    landmarks_mesh1 = TriMesh(mesh1.vertlist[landmarks[:,0]])
    landmarks_mesh2 = TriMesh(mesh2.vertlist[landmarks[:,1]])

    print('Retrieved matching faces from CT & RGBD pointclouds')
    
else:
     landmarks = []

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)

if loadLandmark:
    color = c=np.random.rand(*landmarks_mesh1.vertlist.shape)
    p1.add_points(landmarks_mesh1.vertlist, c = color, shading={"point_size": 0.3})
    p2.add_points(landmarks_mesh2.vertlist, c = color, shading={"point_size": 0.3})


**Generate functional map directly from landmarks**

In [ ]:
# Initialize vector 1 to 5
p2p_21 = np.array([0,1,2,3,4,5])
#p2p_21 = landmarks[:,1]
dim = len(landmarks[:,0])

resultingFM = FunctionalMapping(mesh1=mesh1, mesh2=mesh2)
resultingFM.mesh1.process(k=6, intrinsic=False, verbose=True);
resultingFM.mesh2.process(k=6, intrinsic=False, verbose=True);


print(f'p2p map  : {p2p_21} \n')
print(f'eigen vectors 1 : {resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim]} \n')
print(f'eigen vectors 2 : {resultingFM.mesh2.eigenvectors[landmarks[:,1], :dim]} \n')
print(f'dim : {dim} \n') 

#test = pyFM.spectral.convert.p2p_to_FM(p2p_21, mesh1, mesh2, use_adj=False, subsample=None, use_ANN=False, n_jobs=1)
import pyFM.spectral.convert
#test = pyFM.spectral.convert.mesh_p2p_to_FM(p2p_21, mesh1, mesh2, dim, [landmarks[:,0], landmarks[:,1]])
test = pyFM.spectral.convert.p2p_to_FM(p2p_21, resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim], resultingFM.mesh2.eigenvectors[landmarks[:,1], :])

# Resuling FM
resultingFM.FM = test
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [ ]:
resultingFM.mesh1.process(k=6, intrinsic=False, verbose=True);
resultingFM.mesh2.process(k=6, intrinsic=False, verbose=True);

resultingFM.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
resultingFM._FM_base = test

resultingFM.zoomout_refine(nit=0, step = 0, verbose=True)
print(resultingFM.FM.shape)

p2p_21_zo = resultingFM.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

**Computing descriptors**

In [ ]:
process_params = {
    'n_ev': (25,25),  # Number of eigenvalues on source and Target
    'landmarks': landmarks,  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'HKS',  # WKS or HKS
}


model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);

**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [ ]:
fit_params = {
    'w_descr': 1e0, #scaling for the descriptor preservation term
    'w_lap': 1e-2, #scaling of the laplacian commutativity term
    'w_dcomm': 1e-1, #scaling of the multiplicative operator commutativity
    'w_orient': 0, #scaling of the orientation preservation term
    #'orient_reversing':True, #Whether to use the orientation reversing term instead of the orientation preservation one
    #'optinit':'zeros' #Initialization

}

model.fit(**fit_params, verbose=True)

**Visualizing the associated point to point map**

In [ ]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [ ]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

**Zoomout**

In [ ]:
model.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
model.zoomout_refine(nit=5, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 5- Exporting the resulting maps

In [ ]:
# check if data_out folder exists, if not create it
if not os.path.exists('data_out'):
    os.makedirs('data_out')

# export point to point mappings if they exist

# initial point to point mapping
if 'p2p_21' in locals():
    # open file in write mode
    with open(r'data_out/p2p_21.txt', 'w') as f_export_init:
        for item in p2p_21:
            # write each item on a new line
            f_export_init.write("%s\n" % item)
        print('Exported initial point to point mapping')

# icp refined point to point mapping
if 'p2p_21_icp' in locals():
    # open file in write mode
    with open(r'data_out/p2p_icp.txt', 'w') as f_export_icp:
        for item in p2p_21_icp:
            # write each item on a new line
            f_export_icp.write("%s\n" % item)
        print('Exported icp refined point to point mapping')

# zoomout refined point to point mapping
# open file in write mode
with open(r'data_out/p2p_zo.txt', 'w') as f_export_zo:
    for item in p2p_21_zo:
        # write each item on a new line
        f_export_zo.write("%s\n" % item)
    print('Exported zoomout refined point to point mapping')
